In [7]:
import nltk
nltk.download('punkt') # The NLTK tokenizer
nltk.download('maxent_ne_chunker') # NLTK named-entity chunker
nltk.download('words') # NLTK list of words
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [8]:
sentence = ('In 2017, star quarterback Tom Brady led the Patriots to the Super Bowl, '
           'but lost to the Philadelphia Eagles.')

# 1. Tokenize using the NLTK tokenizer.
tokens = nltk.word_tokenize(sentence)

# 2. Tag parts of speech using the NLTK part-of-speech tagger.
tagged = nltk.pos_tag(tokens)

# 3. Extract entities from tagged sentence.
entities = nltk.chunk.ne_chunk(tagged)
print(entities)

(S
  In/IN
  2017/CD
  ,/,
  star/NN
  quarterback/NN
  (PERSON Tom/NNP Brady/NNP)
  led/VBD
  the/DT
  (ORGANIZATION Patriots/NNP)
  to/TO
  the/DT
  (ORGANIZATION Super/NNP Bowl/NNP)
  ,/,
  but/CC
  lost/VBD
  to/TO
  the/DT
  (ORGANIZATION Philadelphia/NNP Eagles/NNP)
  ./.)


In [9]:
named_entities = [entity for entity in entities if isinstance(entity, nltk.tree.Tree)]
print(named_entities)

[Tree('PERSON', [('Tom', 'NNP'), ('Brady', 'NNP')]), Tree('ORGANIZATION', [('Patriots', 'NNP')]), Tree('ORGANIZATION', [('Super', 'NNP'), ('Bowl', 'NNP')]), Tree('ORGANIZATION', [('Philadelphia', 'NNP'), ('Eagles', 'NNP')])]


In [10]:
import functools

@functools.lru_cache(maxsize=2**14)
def get_entities(sentence):
    tokens = nltk.word_tokenize(sentence)
    tagged = nltk.pos_tag(tokens)
    # Setting `binary=True` makes NLTK return all of the named
    # entities tagged as NNP instead of detailed tags like
    #'Organization', 'Geo-Political Entity', etc.
    entities = nltk.chunk.ne_chunk(tagged, binary=True)
    return entities.leaves()

In [11]:
sentence = 'Jack Black starred in the 2003 film classic "School of Rock".'
get_entities(sentence)

[('Jack', 'NNP'),
 ('Black', 'NNP'),
 ('starred', 'VBD'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('2003', 'CD'),
 ('film', 'NN'),
 ('classic', 'JJ'),
 ('``', '``'),
 ('School', 'NNP'),
 ('of', 'IN'),
 ('Rock', 'NNP'),
 ("''", "''"),
 ('.', '.')]

In [17]:
from textattack.constraints import Constraint

class NamedEntityConstraint(Constraint):
    """ A constraint that ensures `transformed_text` only substitutes named entities from `current_text` with other named entities.
    """
    def _check_constraint(self, transformed_text, current_text):
        transformed_entities = get_entities(transformed_text.text)
        current_entities = get_entities(current_text.text)
        # If there aren't named entities, let's return False (the attack
        # will eventually fail).
        if len(current_entities) == 0:
            return False
        if len(current_entities) != len(transformed_entities):
            # If the two sentences have a different number of entities, then 
            # they definitely don't have the same labels. In this case, the 
            # constraint is violated, and we return False.
            return False
        else:
            # Here we compare all of the words, in order, to make sure that they match.
            # If we find two words that don't match, this means a word was swapped 
            # between `current_text` and `transformed_text`. That word must be a named entity to fulfill our
            # constraint.
            current_word_label = None
            transformed_word_label = None
            for (word_1, label_1), (word_2, label_2) in zip(current_entities, transformed_entities):
                if word_1 != word_2:
                    # Finally, make sure that words swapped between `x` and `x_adv` are named entities. If 
                    # they're not, then we also return False.
                    if (label_1 not in ['JJ', 'CD', 'DT']) or (label_2 not in ['JJ', 'CD', 'DT']):
                        return False            
            # If we get here, all of the labels match up. Return True!
            return True
    

In [18]:
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/albert-base-v2-ag-news")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/albert-base-v2-ag-news")

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Create the goal function using the model
from textattack.goal_functions import UntargetedClassification
goal_function = UntargetedClassification(model_wrapper)

# Import the dataset
from textattack.datasets import HuggingFaceDataset
dataset = HuggingFaceDataset("ag_news", None, "test")

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)
100%|██████████| 2/2 [00:00<00:00, 812.38it/s]
textattack: Loading datasets dataset ag_news, split test.


In [19]:
from textattack.transformations import WordSwapEmbedding
from textattack.search_methods import GreedyWordSwapWIR
from textattack import Attack
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification

# We're going to the `WordSwapEmbedding` transformation. Using the default settings, this
# will try substituting words with their neighbors in the counter-fitted embedding space. 
transformation = WordSwapEmbedding(max_candidates=20) 

# We'll use the greedy search with word importance ranking method again
search_method = GreedyWordSwapWIR()

# Our constraints will be the same as Tutorial 1, plus the named entity constraint
constraints = [RepeatModification(),
               StopwordModification(),
               NamedEntityConstraint(False)]

# Now, let's make the attack using these parameters. 
attack = Attack(goal_function, constraints, transformation, search_method)

In [20]:
from textattack.loggers import CSVLogger # tracks a dataframe for us.
from textattack.attack_results import SuccessfulAttackResult
from textattack import Attacker, AttackArgs

attack_args = AttackArgs(num_successful_examples=5, log_to_csv="results.csv", csv_coloring_style="html")
attacker = Attacker(attack, dataset, attack_args)

attacker.attack_dataset()

textattack: Logging to CSV at path results.csv


Attack2(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  20
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): NamedEntityConstraint(
        (compare_against_original):  False
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   0%|          | 0/5 [00:02<?, ?it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Business (75%)]] --> [[[FAILED]]]

Fears for T N pension after talks Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.




[Succeeded / Failed / Skipped / Total] 0 / 2 / 0 / 2:   0%|          | 0/5 [00:09<?, ?it/s]

...

--------------------------------------------- Result 159 ---------------------------------------------
[[Sports (100%)]] --> [[[FAILED]]]

Phelps Eyes Fourth Gold  ATHENS (Reuters) - A weary Michael Phelps targeted his  fourth Olympic gold medal in Athens, turning his attention on  Wednesday to the 200 meters individual medley and settling for  the second-fastest overall time in the heats.




[Succeeded / Failed / Skipped / Total] 4 / 146 / 10 / 160:  80%|████████  | 4/5 [13:18<03:19, 199.68s/it]

--------------------------------------------- Result 160 ---------------------------------------------
[[World (100%)]] --> [[[FAILED]]]

Israel Kills 5 in Attempt to Assassinate Hamas Man  GAZA (Reuters) - A senior Hamas leader survived an Israeli  assassination attempt in the Gaza Strip Wednesday but at least  five other Palestinians were killed in the explosion that tore  through his home.




[Succeeded / Failed / Skipped / Total] 5 / 146 / 10 / 161: 100%|██████████| 5/5 [13:21<00:00, 160.25s/it]

--------------------------------------------- Result 161 ---------------------------------------------
[[Sports (89%)]] --> [[World (55%)]]

Giants win 6th straight, but Schmidt is injured SAN FRANCISCO -- With the [[first]] doubleheader at SBC Park set to go off, today already stood to be a [[long]] workday for the Giants. It will come on the heels of an even longer night. 

Giants win 6th straight, but Schmidt is injured SAN FRANCISCO -- With the [[fiirst]] doubleheader at SBC Park set to go off, today already stood to be a [[interminable]] workday for the Giants. It will come on the heels of an even longer night. 



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 5      |
| Number of failed attacks:     | 146    |
| Number of skipped attacks:    | 10     |
| Original accuracy:            | 93.79% |
| Accuracy under attack:        | 90.68% |
| Attack success rate:      

In [16]:
import pandas as pd
pd.options.display.max_colwidth = 480 # increase column width so we can actually read the examples

from IPython.core.display import display, HTML

logger = attacker.attack_log_manager.loggers[0]
successes = logger.df[logger.df["result_type"] == "Successful"]
display(HTML(successes[['original_text', 'perturbed_text']].to_html(escape=False)))

/tmp/ipykernel_29018/2781339431.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


,original_text,perturbed_text
0,Fears for T N pension after talks Unions representing workers at Turner Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.,Fears for T N pension after talks Unions representing workers at Knapp Newall say they are 'disappointed' after talks with stricken parent firm Federal Titan.
3,"Prediction Unit Helps Forecast Wildfires (AP) AP - It's barely dawn when Mike Fitzpatrick starts his shift with a blur of colorful maps, figures and endless charts, but already he knows what the day will bring. Lightning will strike in places he expects. Winds will pick up, moist places will dry and flames will roar.","Foresight Driving Helps Expectations Wildfires (AP) AP - It's barely dawn when Meek Fitzpatrick starts his shift with a blur of colorful maps, figures and endless charts, but already he knows what the day will bring. Lightning will strike in places he expects. Winds will pick up, moist places will dry and flames will roar."
8,"E-mail scam targets police chief Wiltshire Police warns about ""phishing"" after its fraud squad chief was targeted.","E-mail scam targets police chief Wiltshire Constabulary warns about ""phishing"" after its fraud squad chief was targeted."
11,"Apple Launches Graphics Software, Video Bundle LOS ANGELES (Reuters) - Apple Computer Inc.<AAPL.O> on Tuesday began shipping a new program designed to let users create real-time motion graphics and unveiled a discount video-editing software bundle featuring its flagship Final Cut Pro software.","Apple Startup Charting Software, Film Pooling LOS FRESNO (Msnbc) - Apple Team Inc.<AAPL.s> on Friday began shipping a new program designed to let users create real-time motion graphics and unveiled a discount video-editing software bundle featuring its flagship Conclude Cuts Careers software."
12,"Dutch Retailer Beats Apple to Local Download Market AMSTERDAM (Reuters) - Free Record Shop, a Dutch music retail chain, beat Apple Computer Inc. to market on Tuesday with the launch of a new download service in Europe's latest battleground for digital song services.","Dutch Retailer Beats Abel to Local Absolution Market AMSTERDAM (Reuters) - Free Registering Depot, a Dutch music retail chain, beat Cobbler Typewriters Inc. to market on Tuesday with the launch of a new download service in Europe's latest battleground for digital song services."
